# Econometric UNFCCC Green Cities Commitment Analysis: USA
## Data Preparation

#### Importing and Cleaning Datasets
Data includes the following:
1. Full list of US City, County and State data
2. UNFCCC data on cities' commitments, actions, etc
3. 2020 Election results by state
4. Land Temperature data
5. US Bureau of Labor Statistics State Unemployment Rate
6. Poverty Estimates for U.S., States, and Counties, 2021

In [1]:
import pandas as pd
import numpy as np
import re

#### 1. US Cities, Counties and State Data
simplemaps.com. US Cities Database | Simplemaps.com. [online] Available at: https://simplemaps.com/data/us-cities.

In [2]:
Cities = pd.read_csv('CityCountyStateMapping.csv')

In [3]:
Cities.head()

,id,city,state,state_name,fips,county,latitude,longitude
0,1840034016,New York City,NY,New York,36081,Queens,40.6943,-73.9249
1,1840020491,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068
2,1840000494,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866
3,1840015149,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101
4,1840020925,Houston,TX,Texas,48201,Harris,29.7860,-95.3885


#### 2. UNFCCC Data
Global Climate Action UNFCCC - Actor Tracking (2022). Available at: https://climateaction.unfccc.int/Actors.

In [4]:
UNFCCC = pd.read_csv('UNFCCC.csv')

In [5]:
UNFCCC = UNFCCC[(UNFCCC['country'] == 'United States of America') & (UNFCCC['organizationType'] == 'City') & (UNFCCC['actorProperties_population'] >= 1000)]  # Filter for cities in the US with populations over 1000 
UNFCCC['Date'] = pd.to_datetime(UNFCCC['Date'], format = '%d/%m/%Y')  # format date as datetime object
UNFCCC = UNFCCC.sort_values(by = 'Date', ascending = False)
UNFCCC = UNFCCC.drop_duplicates(subset = 'publicId', keep = 'first')  # keep only most recent observation of a city
UNFCCC.reset_index(inplace = True)  # Reset index
UNFCCC.drop(['index','Date','id','accountingYear','organizationType'], axis = 1, inplace = True)  # drop unneeded columns
UNFCCC.rename(columns = {'actorProperties_population':'population'}, inplace = True)

In [6]:
# Generate City and State Columns from OrganisationName
UNFCCC['city'] = UNFCCC['organizationName'].apply(lambda x: x.split(',')[0].strip())
UNFCCC['state'] = UNFCCC['organizationName'].apply(lambda x: x.split(',')[1].strip() if len(x.split(',')) == 2 else np.NaN)   

# Clean City names for better joining
strings_to_remove = ['City of', 'City Of', 'Town of', 'Town Of', 'Township Of', 'City and County of', 'Borough Of', 'Village of', '(Town)']
for string in strings_to_remove:
    UNFCCC['city'] = UNFCCC['city'].str.replace(string, '', regex = False)

UNFCCC['city'] = UNFCCC['city'].apply(lambda x: x.strip())

UNFCCC = UNFCCC[['city','state','population','hasCommitments','hasActionsUndertaken',
                'hasEmissionInventory','hasInitiativeParticipations','hasImpact','hasMitigations',
                'hasAdaptations','hasRiskAssessments','hasClimateActionPlans','hasFinanceActions']]

Remove the following irregular / nonconforming observations:
1. Metropolitan Council, Twin Cities: covers bistate area and Minneapolis is already included
2. Chicago Metropolitan Mayors Caucus:  Chicago is already included
3. Metropolitan Washington Council of Governments (COG):  covers bistate area and District of Columbia is already included
4. Mid-America Regional Council:  covers bistate area and Kansas City is already included
5. San Francisco/Bay Area Air Quality Management District:  San Francisco is already included

In [7]:
cities_to_remove = ['Metropolitan Council', 'Chicago Metropolitan Mayors Caucus', 'Metropolitan Washington Council of Governments (COG)', 'Mid-America Regional Council', 'San Francisco/Bay Area Air Quality Management District’S']
UNFCCC = UNFCCC[~UNFCCC['city'].isin(cities_to_remove)]

Make the following alterations to irregular / nonconforming observations:
1. Metropolitan Government of Nashville and Davidson County, TN: rename to Nashville
2. Durham: add North Carolina as state

In [8]:
UNFCCC.loc[UNFCCC['city'] == 'Metropolitan Government of Nashville and Davidson County', 'city'] = 'Nashville'
UNFCCC.loc[UNFCCC['city'] == 'Durham', 'state'] = 'NC'

In [9]:
UNFCCC.head()

,city,state,population,hasCommitments,hasActionsUndertaken,hasEmissionInventory,hasInitiativeParticipations,hasImpact,hasMitigations,hasAdaptations,hasRiskAssessments,hasClimateActionPlans,hasFinanceActions
0,Park Forest,IL,21261.0,True,True,False,False,False,True,True,True,True,False
1,Emeryville,CA,12905.0,True,True,True,True,False,True,True,True,True,False
2,Grand Rapids,MI,199417.0,True,True,False,True,False,True,True,True,False,False
3,Fremont,CA,230504.0,True,True,True,True,False,True,True,True,True,False
4,Fort Worth,TX,918915.0,False,True,False,False,False,False,True,False,False,False


#### 3. 2020 Election Results by State
Wikipedia Contributors (2019). 2020 United States presidential election. [online] Wikipedia. Available at: https://en.wikipedia.org/wiki/2020_United_States_presidential_election.

In [10]:
ElectionbyState = pd.read_csv('ElectionbyState.csv')

In [11]:
RedBlue = lambda row: 'blue' if row['Biden/Harris_Democratic_EV'] > row['Trump/Pence_Republican_EV'] else 'red'  # Assign blue for a state if it had more democratic electoral college votes
ElectionbyState['redBlueState'] = ElectionbyState.apply(RedBlue, axis = 1)
ElectionbyState = ElectionbyState[['state_abb','redBlueState']]
ElectionbyState.rename(columns = {'state_abb':'state'}, inplace = True)

In [12]:
ElectionbyState.head()

,state,redBlueState
0,AL,red
1,AK,red
2,AZ,blue
3,AR,red
4,CA,blue


#### 4. Land Temperatures from 1828 to 2013
www.kaggle.com. (Berkeley Earth). Climate Change: Earth Surface Temperature Data. [online] Available at: https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data.

In [13]:
LandTemp = pd.read_csv('GlobalLandTemperatures.csv')

In [14]:
LandTemp = LandTemp[LandTemp['Country'] == 'United States']  # filter for USA State Data
LandTemp['Date'] = pd.to_datetime(LandTemp['dt'], format = '%Y-%m-%d')  # create datetime column
LandTemp['Year'] = LandTemp['Date'].dt.year  # create year column
LandTemp['Month'] = LandTemp['Date'].dt.month_name()  # create month column
LandTemp = LandTemp[['Year','Month','State','AverageTemperature']]

In [15]:
LandTemp = LandTemp[LandTemp['Month'].isin(['January','July'])]  # keep only January and July observations
LandTemp = LandTemp.pivot(index = ['Year','State'], columns = 'Month', values = 'AverageTemperature').reset_index()  # Pivot to get widened form
LandTemp['janJulyDiff'] = LandTemp['July'] - LandTemp['January']  # calculate July to January temperature difference

In [16]:
LandTemp[['Year','State','janJulyDiff']].head()

Month,Year,State,janJulyDiff
0,1744,Alabama,NaN
1,1744,Connecticut,NaN
2,1744,Delaware,NaN
3,1744,District Of Columbia,NaN
4,1744,Florida,NaN


#### 5. US Bureau of Labor Statistics State Unemployment Rate (February 2024)
Bls.gov. (2024). Table 1. Civilian labor force and unemployment by state and selected area, seasonally adjusted. [online] Available at: https://www.bls.gov/news.release/laus.t01.htm.

In [17]:
Unemp = pd.read_csv('StateEmployment.csv')

In [18]:
Unemp.head()

,state,unemploymentRate
0,AK,0.047
1,AL,0.030
2,AR,0.036
3,AZ,0.041
4,CA,0.053


#### 6. Poverty Estimates for U.S., States, and Counties, 2021
U.S. Department of Commerce, Bureau of the Census, Small Area Income and Poverty Estimates (SAIPE) Program

In [19]:
Poverty = pd.read_csv('PovertyEstimates.csv')

In [20]:
Poverty['povertyProp'] = Poverty['PCTPOVALL_2021'] / 100  # convert to decimal
Poverty = Poverty[['county','state','fips','povertyProp']]

In [21]:
Poverty.head()

,county,state,fips,povertyProp
0,United States,US,0,0.128
1,Alabama,AL,1000,0.163
2,Autauga County,AL,1001,0.107
3,Baldwin County,AL,1003,0.108
4,Barbour County,AL,1005,0.230


#### Joining Datasets

In [22]:
t = pd.merge( UNFCCC,Cities, how = 'left', on = ['state','city'])
t[t.isna().any(axis=1)]

,city,state,population,hasCommitments,hasActionsUndertaken,hasEmissionInventory,hasInitiativeParticipations,hasImpact,hasMitigations,hasAdaptations,hasRiskAssessments,hasClimateActionPlans,hasFinanceActions,id,state_name,fips,county,latitude,longitude
64,Broward County,FL,1930983.0,True,True,True,True,False,True,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN
65,Boulder County,CO,324682.0,True,True,True,False,False,True,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN
67,Abington Township,PA,58502.0,False,True,True,False,False,True,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN
96,District of Columbia,DC,690093.0,True,True,True,True,False,True,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN
97,Dane County,WI,563951.0,True,True,False,True,False,True,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN
98,Cuyahoga County,OH,1253889.0,True,True,True,True,False,True,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN
118,Township of Maplewood,NJ,25500.0,False,True,False,False,False,True,True,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
119,York,ME,13161.0,True,True,True,True,False,True,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN
122,Lexington,MA,33340.0,True,True,True,True,False,True,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN
126,Acton,MA,23662.0,True,True,False,True,False,False,True,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN
